In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
import os
import cv2
import numpy as np
import json

num_classes = 96

input_shape = (448, 448, 3)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = BatchNormalization()(x)
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.35)(x)

bbox_output = Dense(4, activation='linear', name='bboxes_output')(x)

labels_output = Dense(num_classes, activation='softmax', name='labels_output')(x)

model = Model(inputs=base_model.input, outputs=[bbox_output, labels_output])

# Загрузка модели
model.load_weights('./results/HCR-Net-Detection.h5')

def load_images_from_directory(directory, image_size):
    images = []
    image_files = []
    for filename in os.listdir(directory):
        if filename.endswith(".png") or filename.endswith(".jpg") or filename.endswith(".bmp"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (image_size, image_size))
            images.append(img)
            image_files.append(filename)
    return np.array(images), image_files

def predict_and_save(model, images, image_files, output_dir):
    predictions = model.predict(images)
    bbox_predictions, class_predictions = predictions

    for i in range(len(images)):
        image_result = {
            "predictions": []
        }

        for bbox, cls in zip(bbox_predictions[i], class_predictions[i]):
            image_result["predictions"].append({
                "class": int(np.argmax(cls)),
                "bbox": [float(coord) for coord in bbox]
            })

        json_filename = os.path.join(output_dir, f'{os.path.splitext(image_files[i])[0]}_pred.json')
        with open(json_filename, 'w') as f:
            json.dump(image_result, f, indent=4)

predict_image_dir = './output/test'
output_dir = './output/test'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

images, image_files = load_images_from_directory(predict_image_dir, 448)

images = images / 255.0

predict_and_save(model, images, image_files, output_dir)

predict_image_dir = './output/train'
output_dir = './output/train'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

images, image_files = load_images_from_directory(predict_image_dir, 448)

images = images / 255.0

predict_and_save(model, images, image_files, output_dir)